In [ ]:
import json
import numpy as np
import pandas as pd
from sys import exit, stderr, stdin, stdout
from time import sleep
# from tqdm import tqdm
from tqdm.notebook import tqdm
from os.path import dirname, join, isfile, isdir
from os import makedirs
import subprocess
from datetime import datetime, timedelta, date

from bulletin.notifica import Notifica
from bulletin.utils.utils import Timer, get_better_notifica
from bulletin.utils.normalize import normalize_hash
from bulletin.utils.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

tqdm.pandas()

timerAll = Timer()
timer = Timer()

In [ ]:
baixar_notifica = False#True if input("Enter para continuar, S para baixar notifica") == 'S' else False
ler_notifica = False#True if input("Enter para continuar, S para ler notifica") == 'S' else False

In [ ]:
notifica = Notifica()
if ler_notifica or baixar_notifica:
    if baixar_notifica:
        notifica.download_todas_notificacoes()
    if ler_notifica:
        notifica.read_todas_notificacoes()
        notifica.save()
else:
    notifica.load()

print(notifica.shape())

In [ ]:
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['mun_resid']=='CURITIBA') & (casosn['classificacao_final']==2)]
casosn.groupby('classificacao_final')[['id']].count().append(pd.DataFrame(index=['total'],columns=['id'],data=[len(casosn)])).to_csv('classificacao_final_count.csv')

In [ ]:
timer.start()

pacientes = { 'pacientes': casosn['paciente'].apply(normalize_hash).values.tolist() }
with open('pacientes.json', 'w') as json_file:
  json.dump(pacientes, json_file)

subprocess.call(("node", 'soundex.js'))

with open('pacientes_soundex.json', 'r') as json_file:
  pacientes = json.load(json_file)

casosn['soundex'] = pacientes
casosn = casosn.sort_values('soundex')

timer.stop()
print('soundex: ',timer.ftime())

In [ ]:
timer.start()

keys = ['soundex','paciente','idade','ibge_unidade_notifica','ibge_residencia','data_nascimento','nome_mae','cpf','id']

casosn_hash = casosn[keys+['status_notificacao','classificacao_final','metodo','evolucao']]
casos_agrupados = casosn_hash.sort_values('soundex').groupby(keys[:-1])
casos_agrupados_count = casos_agrupados[['id']].count().rename(columns={'id':'count'}).sort_values('count', ascending=False)
casos_agrupados_count.to_csv('casos_agrupados_count.csv')

timer.stop()
print('casos_agrupados_count: ',timer.ftime())

In [ ]:
timer.start()

pacientes = pd.DataFrame(index=casos_agrupados_count.index,columns=['ids','status_notificacao','classificacao_final','metodo','evolucao'], data=[])

for idx,df in casos_agrupados:
    pacientes.loc[idx,'ids'] = [ x for x in df['id'].values ]
    pacientes.loc[idx,'status_notificacao'] = [ x for x in df['status_notificacao'].values ]
    pacientes.loc[idx,'classificacao_final'] = [ x for x in df['classificacao_final'].values ]
    pacientes.loc[idx,'metodo'] = [ x for x in df['metodo'].values ]
    pacientes.loc[idx,'evolucao'] = [ x for x in df['evolucao'].values ]

pacientes.to_csv('pacientes.csv')
timer.stop()
print(f'pacientes ({len(pacientes)}): ',timer.ftime())

In [ ]:
timer.start()

pacientes_duplicados = pacientes.loc[ [True if len(ids) > 1 else False for ids in pacientes['ids']]]
pacientes_duplicados.to_csv('pacientes_duplicados.csv')

timer.stop()
print(f'pacientes_duplicados ({len(pacientes_duplicados)}): ',timer.ftime())

In [ ]:
exit()
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    #check reinfection
    manter.append(idx)


casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_diag'].notnull()) & (casosn.duplicated('hash_diag',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_diag: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_diag'))} pacientes pelo hash_diag que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_diag'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_hash_diag.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
notifica.save(casosn)